In [1]:
# parse dev data function
def ParseJson(data_json):
    paragraphs = data_json['data'][0]['paragraphs']
    idx_list = list()
    context_list = list()
    ques_list = list()
    ans_list = list()
    for para in paragraphs:
        context = para['context']
        idx = para['qas'][0]['id']
        ques = para['qas'][0]['question']
        ans = '|'.join([ans['text'] for ans in para['qas'][0]['answers']])
        idx_list.append(idx)
        context_list.append(context)
        ques_list.append(ques)
        ans_list.append(ans)
    return idx_list, context_list, ques_list, ans_list

In [3]:
# dev data
import json
dev_file = '/home/t-honli/data/EQnA/dev-v1.1.json' 
dev_data = json.load(open(dev_file, "r"))

In [4]:
import pandas as pd
idx, context, ques, ans = ParseJson(dev_data)
dev_pd = pd.DataFrame({'id':idx, 'context':context, 'question':ques, 'ground_truth':ans}, columns=['id', 'context', 'question', 'ground_truth'])
dev_pd.head(5)

,id,context,question,ground_truth
0,2abdda92eac7f6b73d9b238f503e4219,Start Up & Rev of a 2006 Acura RL SH - AWD.Thi...,2006 acura rl horsepower,290 HP
1,f26402275d1f070d15e951a3ec04dacb,The income tax withholding rate remains at 4.2...,2015 michigan withholding tax rate,4.25 %
2,125674965d00b4087f076223745f1ad5,The experts at Vidal Sassoon bring you salon c...,5rr hair color,Red
3,b153122979cdb753b1466ac8145758c9,Effects depend on the substance : hydrogen per...,acid that burns through skin,nitric acid|hydrogen peroxide
4,f5d4a08ddf0aba6cab1ea9ae6bd1f26b,"Actor Nikolaj Coster - Waldau , the man who pl...",actor who plays jamie lannister,Nikolaj Coster|Waldau


In [21]:
# bidaf ans fact
import json
import os
file = '/home/t-honli/bi-att-flow/out/EQnA_retrain/answer/test-108000.json'
bidaf_Fact = json.load(open(file, "r"))

In [22]:
dev_pd['bidaf_Fact'] = dev_pd.apply(lambda row: bidaf_Fact[row['id']], axis=1)

In [15]:
# bidaf ans EQnA
import json
import os
file = '/home/t-honli/bi-att-flow/out/EQnA/no_sent_token/01-08-2017/answer/test-020000.json'
bidaf_EQnA = json.load(open(file, "r"))
dev_pd['bidaf_EQnA'] = dev_pd.apply(lambda row: bidaf_EQnA[row['id']], axis=1)

In [19]:
# prod model
import pandas as pd
col_names = ['id', 'Query', 'Url', 'Answer', 'AnswerTokenList', 'ParaseSpan', 'phrase', 'Label', 'Probability']
prod = pd.read_csv('/home/t-honli/data/squad_EQnA/EQnA_Highlighting_Test_ProdModel.tsv', header=None, sep='\t', names=col_names, dtype=str).fillna('')

print (len(prod))

# 根据answer，query，url生成hash_id
import hashlib
def GetHashCode(context):
    hash = hashlib.md5()
    hash.update(context.encode('utf-8'))
    return hash.hexdigest()
prod['hash_id'] = prod.apply(lambda row: GetHashCode(row['Answer'] + ' ' + row['Query'] + ' ' + row['Url']), axis=1)

prod = prod[prod.apply(lambda row: len(row['phrase'])<32, axis=1)]
print (len(prod))
prod.head(5)

15560
15454


,id,Query,Url,Answer,AnswerTokenList,ParaseSpan,phrase,Label,Probability,hash_id
0,00021900a221a0b2e5c9a923c4f13813,where does the water from mississippi's rivers...,http://en.wikipedia.org/wiki/Mississippi_River,"In the United States, the Mississippi River dr...","[""In"",""the"",""United"",""States"","","",""the"",""Missi...",79:79,rivers,0,0.01860236,9b6bef3c4ce179302bbad6ddf9b98832
1,0004dd703e44213707c923e548627774,good foods for pregnant women,http://www.netdoctor.co.uk/health_advice/facts...,"During pregnancy, a woman's body needs more ir...","[""During"",""pregnancy"","","",""a"",""woman"",""'s"",""bo...",4:4,woman,0,0.1246661,ac6273ec9adf9778fb33f09501f43874
2,00052a47dccf1667995ebffb3019b565,what continent is belize on,http://www.belize.com/where-is-belize,Belize is located on the Caribbean coast of no...,"[""Belize"",""is"",""located"",""on"",""the"",""Caribbean...",19:19,88°45,0,0.02859148,2110154eb61df6e89f85c33c204492e0
3,00099640760fac2092756da3714c60a9,when was tom seaver born,http://www.baseball-almanac.com/players/player...,"Tom Seaver was born on Friday, November 17, 19...","[""Tom"",""Seaver"",""was"",""born"",""on"",""Friday"","",""...",17:17,Seaver,0,0.001268676,a272a8fdd6b0e63d75e39f732639c31f
4,000e8ade636d16933ddc53486a59a976,disease caused by deer ticks,http://www.niaid.nih.gov/topics/tickborne/Page...,"Ticks transmit ehrlichiosis and anaplasmosis, ...","[""Ticks"",""transmit"",""ehrlichiosis"",""and"",""anap...",0:0,Ticks,0,0.008481221,54ee3a3b1b57174ee8ee50625d20ee74


In [23]:
# 获取hash_id, [(phrase,score)]
multi_phrase_dict = {}
def getPhrase(row):
    if row['hash_id'] not in multi_phrase_dict:
        multi_phrase_dict[row['hash_id']] = list()
    multi_phrase_dict[row['hash_id']].append(row['phrase'] + ":" + row['Probability'])
prod.apply(getPhrase, axis=1)
print (len(prod))

15454


In [43]:
# sort
for idx in list(multi_phrase_dict.keys()):
    multi_phrase_dict[idx].sort(key=lambda x: float(x.split(':')[-1]), reverse=True)

In [44]:
print (list(multi_phrase_dict.items())[:5])

[('9b6bef3c4ce179302bbad6ddf9b98832', ['Alabama:0.8559828', 'Rio Grande:0.7438805', 'Saint Lawrence River:0.7311112', 'Tombigbee:0.6697654', 'Chattahoochee:0.5843887', 'the Atlantic Ocean:0.4905324', 'Red River of the North:0.4180078', 'the United States:0.3122597', 'the Great Lakes:0.305788', 'waterways:0.2183529', 'the Mississippi:0.07368357', 'drained:0.04473107', 'coastal:0.02576688', 'rivers:0.02443846', 'rivers:0.01860236', 'regions:0.01731697']), ('ac6273ec9adf9778fb33f09501f43874', ['spinach:0.9554551', 'broccoli:0.94595', 'green vegetables:0.9092412', 'wholemeal bread:0.8100643', 'blood:0.6137482', 'produce:0.5851527', 'iron:0.4314162', 'placenta:0.2960912', 'iron:0.2656902', 'nutrition:0.2498231', 'pregnancy:0.1734539', 'woman:0.1246661', 'body:0.06024067']), ('2110154eb61df6e89f85c33c204492e0', ['South America:0.9047099', 'North America:0.6378803', 'Yucatán Peninsula:0.1944177', 'the Caribbean:0.1486141', 'Prime Meridian:0.1198179', 'the equator:0.0431998', '17°15:0.02910043

In [45]:
dev_pd['prod'] = dev_pd.apply(lambda row: '|'.join(multi_phrase_dict[row['id']]), axis=1)

In [7]:
pd.set_option('max_colwidth', 200)
dev_pd.head(5)

,id,context,question,ground_truth,bidaf
0,2abdda92eac7f6b73d9b238f503e4219,"Start Up & Rev of a 2006 Acura RL SH - AWD.This has a 3.5L V6 making a decent 290 HP while paired to a 5 Speed Auto.The interior in this was very classy , roomy , & Comfortable .",2006 acura rl horsepower,290 HP,290 HP
1,f26402275d1f070d15e951a3ec04dacb,"The income tax withholding rate remains at 4.25 % for tax year 2015 . However , the personal exemption amount for tax year 2015 will change to $4,000 . You may continue to use 2014 Michigan Income...",2015 michigan withholding tax rate,4.25 %,4.25 %
2,125674965d00b4087f076223745f1ad5,The experts at Vidal Sassoon bring you salon color expertise with this Vidal Sassoon Pro Series Hair Color in 5RR Medium Vibrant Red . It offers 100 percent gray coverage in a permanent color form...,5rr hair color,Red,Red
3,b153122979cdb753b1466ac8145758c9,"Effects depend on the substance : hydrogen peroxide removes a bleached layer of skin , while nitric acid causes a characteristic color change to yellow in the skin . Chemical burns may occur throu...",acid that burns through skin,nitric acid|hydrogen peroxide,hydrogen peroxide
4,f5d4a08ddf0aba6cab1ea9ae6bd1f26b,"Actor Nikolaj Coster - Waldau , the man who plays Jaime Lannister in HBO 's popular television series Game Of Thrones , has been spotted in Hobart , where he has spent time taking in the city 's c...",actor who plays jamie lannister,Nikolaj Coster|Waldau,Nikolaj Coster - Waldau


In [29]:
# Fact Good Answer
def getGoodAnswer(row):
    ans_list = map(lambda x: x.strip(), row['ground_truth'].split('|'))
    for ans in ans_list:
        if(row['bidaf_Fact'] == ans):
            return True
    return False
Fact_GoodAnswer_row = dev_pd.apply(getGoodAnswer, axis=1)
Fact_GoodAnswer = dev_pd[Fact_GoodAnswer_row]
Fact_BadAnswer = dev_pd[Fact_GoodAnswer_row.apply(lambda x: not x)]
print (len(Fact_GoodAnswer))
print (len(Fact_BadAnswer))

1564
457


In [40]:
Fact_BadAnswer.to_csv('./Fact_VS_EQnA/Fact_BadAnswer.tsv', header=True, index=None, sep='\t')
Fact_BadAnswer.head(5)

,id,context,question,ground_truth,bidaf_EQnA,bidaf_Fact
4,f5d4a08ddf0aba6cab1ea9ae6bd1f26b,"Actor Nikolaj Coster - Waldau , the man who plays Jaime Lannister in HBO 's popular television series Game Of Thrones , has been spotted in Hobart , where he has spent time taking in the city 's c...",actor who plays jamie lannister,Nikolaj Coster|Waldau,Waldau,Nikolaj Coster - Waldau
14,cecc909419f05700b7fc9ae42e8f1e99,"It is almost exclusively caused by fluid buildup around the eyes , or periorbital edema . Minor puffiness usually detectable below the eyes only ( although at times they could be present all aroun...",area below the eye,eye bags,periorbital edema,periorbital edema
23,d55c72fc739c6fc0ec14e7c7f3a316b4,"For example , an art director ’s average annual wage in 2013 was $96,650 , compared with an architect ’ s average wage of $79,650 and a public relations specialist ’ s average wage of $63,020 in t...",average salary for an art director,"$96,650","$96,650","$63,020"
30,5228e36effb4c9834904b5550d98eb3c,"In some cases , hypochondriasis responds well to antipsychotics , particularly the newer atypical antipsychotic medication .",best medication for hypochondria,atypical antipsychotic|antipsychotics,antipsychotic,antipsychotic medication
31,cdd61c37899920b851bb0565e2778d96,"The only effective treatment for dehydration is to replace lost fluids and lost electrolytes . The best approach to dehydration treatment depends on age , the severity of dehydration and its cause .",best treatment for dehydration,electrolytes,electrolytes,electrolytes . The best approach to dehydration


In [31]:
# EQnA Good Answer
def getGoodAnswer(row):
    ans_list = map(lambda x: x.strip(), row['ground_truth'].split('|'))
    for ans in ans_list:
        if(row['bidaf_EQnA'] == ans):
            return True
    return False
EQnA_GoodAnswer_row = dev_pd.apply(getGoodAnswer, axis=1)
EQnA_GoodAnswer = dev_pd[EQnA_GoodAnswer_row]
EQnA_BadAnswer = dev_pd[EQnA_GoodAnswer_row.apply(lambda x: not x)]
print (len(EQnA_GoodAnswer))
print (len(EQnA_BadAnswer))

1579
442


In [39]:
EQnA_BadAnswer.to_csv('./Fact_VS_EQnA/EQnA_BadAnswer.tsv', header=True, index=None, sep='\t')
EQnA_BadAnswer.head(5)

,id,context,question,ground_truth,bidaf_EQnA,bidaf_Fact
2,125674965d00b4087f076223745f1ad5,The experts at Vidal Sassoon bring you salon color expertise with this Vidal Sassoon Pro Series Hair Color in 5RR Medium Vibrant Red . It offers 100 percent gray coverage in a permanent color form...,5rr hair color,Red,gray,Red
11,a1080f4c349861db9417b92c4b3e4f4c,"The daily value for fiber is 25 grams per day , with this fiber including a mix of soluble and insoluble fibers . A medium banana contains 3.1 grams of fiber , including 1 gram of soluble fiber .",amount of fiber in a banana,3.1 grams|1 gram,25 grams,3.1 grams
14,cecc909419f05700b7fc9ae42e8f1e99,"It is almost exclusively caused by fluid buildup around the eyes , or periorbital edema . Minor puffiness usually detectable below the eyes only ( although at times they could be present all aroun...",area below the eye,eye bags,periorbital edema,periorbital edema
19,11921f3f2454758fd5e091f2fa7e5f5f,"People under 20 years of age made up over a quarter of the U.S. population ( 27.3 % ) , and people age 65 and over made up one - eighth ( 12.8 % ) in 2009 . The national median age was 36.8 years .",average age in united states by year,36.8 years,age 65,36.8 years
30,5228e36effb4c9834904b5550d98eb3c,"In some cases , hypochondriasis responds well to antipsychotics , particularly the newer atypical antipsychotic medication .",best medication for hypochondria,atypical antipsychotic|antipsychotics,antipsychotic,antipsychotic medication


In [48]:
dev_pd.to_csv('./sample.tsv', header=True, index=None, sep='\t')

In [47]:
import numpy as np
x = np.array([])
print (x.shape)
x = [[0 for i in range(300)]]
y = x + [[0 for i in range(300)]]
print (x)
print (y)

(0,)
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 